# M3 Project

In this project, you will compare the performance of uninformed and informed search algorithms for the 4-sided dominoes problem. You will be given N<sup>2</sup> tiles (N = {2,3,4}) and will be asked to arrange them in an NxN grid in a way that adjacent tiles have the same number in their neighboring sides. The tiles cannot be rotated. See the example below for N=3.

<img src="m3project.png" width="400"/>

An initial version of the code with the problem specification (below) and a report template (at the bottom) are available in this notebook. Deliverables are the final code (non-functioning code is worth 0 points) and the comparison report.

Solve the task above using:
- one uninformed search algorithm of your choice (20pts)
- one informed search algorithm of your choice (20pts)

For your solution, describe the:
- search state space (10pts)
- successor function (10pts)
- heuristic function for the informed search (10pts)

Run each algorithm with at least 10 different initial states for each value of N, and compare the performance of the chosen algorithms for different puzzle sizes in terms of:
- average number of expanded states (15pts)
- success rate (15pts)

You are free to set a maximum number of expanded states for each algorithm, and return failure in case this number is reached.

# Implementation

You are free to change the code below as needed.

In [12]:
import random
import heapq

class Domino():
    """
    Implementation of a 4-sided domino tile.

    Methods
    -------
    is_above(other)
        Checks if the tile is above the other tile.
    is_under(other)
        Checks if the tile is under the other tile.
    is_on_the_left_of(other)
        Checks if the tile is on the left of the other tile.
    is_on_the_right_of(other)
        Checks if the tile is on the right of the other tile.
    """
    def __init__(self, top: int, right: int, bottom: int, left: int):
        assert isinstance(top, int) and isinstance(right, int) and isinstance(bottom, int) and isinstance(left, int), "Invalid tile value!"
        self.top = top
        self.right = right
        self.bottom = bottom
        self.left = left

    def is_above(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.bottom == other.top
    
    def is_under(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.top == other.bottom
    
    def is_on_the_left_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.right == other.left
    
    def is_on_the_right_of(self, other):
        assert isinstance(other, Domino), "Invalid tile type!"
        return self.left == other.right
    
    def __lt__(self, other):
        """
        Less than comparison method for Domino objects.

        Parameters
        ----------
        other : Domino
            The other Domino object to compare with.

        Returns
        -------
        bool
            True if self is less than other, False otherwise.
        """
        return (self.top, self.right, self.bottom, self.left) < (other.top, other.right, other.bottom, other.left)


class FourDominoes():
    """
    Implementation of the 4-sided dominoes puzzle.

    Methods
    -------
    show()
        Visualize the current state.
    move(action)
        Apply an action to the current state.
    successor(state)
        Finds the list of successors for a given state.
    goal_test(state)
        Checks if the current state is a goal state.
    """

    def __init__(self, N, state=None):
        """
        Parameters
        ----------
        N : int
            Grid size (puzzle contains N^2 tiles)
        state : tuple, optional
            Initial state configuration. If None is provided, a random one is created.
        """
        assert N >= 2 and N <= 4, "Invalid grid size!"
        self.N = N

        if state is not None:
            assert len(state) == self.N, "Invalid state size!"
            for row in state:
                assert len(row) == self.N, "Invalid state size!"
                for tile in row:
                    assert isinstance(tile, Domino), "Invalid state type!"
            self.state = state
        else:
            self.state = self.__get_random_state()
    
    def __get_random_state(self):
        """
        Generates a random puzzle configuration (grid of dominoes)

        Returns
        -------
        tuple
            A tuple describing a unique puzzle configuration.
        """
        temp = []
        for i in range(self.N):
            for j in range(self.N):
                domino = Domino(
                    random.randint(1, 9) if i == 0 else temp[(i - 1) * self.N + j].bottom,  # top
                    random.randint(1, 9),                                                  # right
                    random.randint(1, 9),                                                  # bottom
                    random.randint(1, 9) if j == 0 else temp[i * self.N + j - 1].right       # left
                )
                temp.append(domino)
        random.shuffle(temp)  # if you comment this line, the state will be a final state (solution)
        return tuple(tuple(temp[i * self.N:(i + 1) * self.N]) for i in range(self.N))

    def show(self):
        """
        Prints the current state.
        """
        # line
        print('╔', end='')
        for i in range(self.N):
            print('═══', end='╦' if i < self.N - 1 else '╗\n')
        for i in range(self.N):
            # first line of tile
            print('║', end='')
            for j in range(self.N):
                print('╲{}╱║'.format(self.state[i][j].top), end='' if j < self.N - 1 else '\n')
            # second line of tile
            print('║', end='')
            for j in range(self.N):
                print('{}╳{}║'.format(self.state[i][j].left, self.state[i][j].right), end='' if j < self.N - 1 else '\n')
            # third line of tile
            print('║', end='')
            for j in range(self.N):
                print('╱{}╲║'.format(self.state[i][j].bottom), end='' if j < self.N - 1 else '\n')
            # line
            print('╠' if i < self.N - 1 else '╚', end='')
            for j in range(self.N):
                print('═══', end='╬' if i < self.N - 1 and j < self.N - 1 else '╩' if i == self.N - 1 and j < self.N - 1 else '╣\n' if i < self.N - 1 else '╝\n')

    def move(self, action):
        """
        Uses a given action to update the current state of the puzzle. Assumes the action is valid.

        Parameters
        ----------
        action : tuple
            Tuple with coordinates of two tiles to be swapped ((row1,col1), (row2,col2))
        """
        assert len(action) == 2 and all(len(coord) == 2 for coord in action) and all(
            isinstance(x, int) and x >= 0 and x < self.N for coord in action for x in coord), "Invalid action!"
        (r1, c1), (r2, c2) = action
        temp = [list(row) for row in self.state]
        temp[r1][c1], temp[r2][c2] = temp[r2][c2], temp[r1][c1]
        self.state = tuple(tuple(x) for x in temp)

    def successor(self, state):
        """
        Finds the list of successors for a given state.

        Parameters
        ----------
        state : tuple
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        list
            A list of pairs (action, state) with all states that can be reached from the given state with a single action.
        """
        successors = []
        for i in range(self.N * self.N):
            r1 = i // self.N
            c1 = i % self.N
            for j in range(i + 1, self.N * self.N):
                r2 = j // self.N
                c2 = j % self.N
                action = ((r1,c1), (r2,c2))
                copy = FourDominoes(self.N, state)
                copy.move(action)
                successors.append((action,copy.state))
        return successors

    def goal_test(self, state):
        """
        Checks if the given state is a goal state.

        Parameters
        ----------
        state : tuple
            A tuple describing a unique puzzle configuration.

        Returns
        -------
        bool
            True if the given state is a goal state, and False otherwise.
        """
        for i in range(self.N):
            for j in range(self.N):
                if i > 0 and not state[i][j].is_under(state[i - 1][j]):
                    return False
                if j > 0 and not state[i][j].is_on_the_right_of(state[i][j - 1]):
                    return False
        return True


def uninformed_search(problem, max_expanded_states):
    """
    Performs uninformed search using a specific strategy.

    Parameters
    ----------
    problem : FourDominoes
        The problem instance to be solved.
    max_expanded_states : int
        Maximum number of expanded states.

    Returns
    -------
    path : list
        A list of actions representing the path from the initial state to the goal state.
    num_expanded : int
        The number of expanded states during the search.
    success : bool
        True if a solution was found, False otherwise.
    """
    num_expanded = 0
    frontier = [(problem.state, [])]  # (state, path)
    explored = set()

    while frontier:
        state, path = frontier.pop(0)
        if problem.goal_test(state):
            return path, num_expanded, True
        explored.add(state)

        if num_expanded >= max_expanded_states:
            return None, num_expanded, False

        successors = problem.successor(state)
        for action, succ_state in successors:
            if succ_state not in explored:
                frontier.append((succ_state, path + [action]))
                num_expanded += 1

    return None, num_expanded, False


def heuristic(state):
    """
    Heuristic function for the informed search algorithm.

    Parameters
    ----------
    state : tuple
        A tuple describing a unique puzzle configuration.

    Returns
    -------
    tuple
        The heuristic value for the given state along with the state itself.
    """
    misplaced_count = 0
    for i in range(len(state)):
        for j in range(len(state)):
            if i > 0 and not state[i][j].is_under(state[i - 1][j]):
                misplaced_count += 1
            if j > 0 and not state[i][j].is_on_the_right_of(state[i][j - 1]):
                misplaced_count += 1
    return misplaced_count, state


def informed_search(problem, max_expanded_states):
    """
    Performs informed search using a specific strategy.

    Parameters
    ----------
    problem : FourDominoes
        The problem instance to be solved.
    max_expanded_states : int
        Maximum number of expanded states.

    Returns
    -------
    path : list
        A list of actions representing the path from the initial state to the goal state.
    num_expanded : int
        The number of expanded states during the search.
    success : bool
        True if a solution was found, False otherwise.
    """
    num_expanded = 0
    frontier = [(heuristic(problem.state), problem.state, [])]  # (heuristic, state, path)
    explored = set()

    while frontier:
        _, state, path = heapq.heappop(frontier)
        if problem.goal_test(state):
            return path, num_expanded, True
        explored.add(state)

        if num_expanded >= max_expanded_states:
            return None, num_expanded, False

        successors = problem.successor(state)
        for action, succ_state in successors:
            if succ_state not in explored:
                heapq.heappush(frontier, (heuristic(succ_state), succ_state, path + [action]))
                num_expanded += 1

    return None, num_expanded, False


def run_experiments():
    puzzle_sizes = [2, 3, 4]
    max_expanded_states = 10000
    num_initial_states = 10

    for N in puzzle_sizes:
        print('-----------------')
        print('{}x{} grid of tiles'.format(N, N))
        print('-----------------')

        total_expanded_states_uninformed = 0
        total_expanded_states_informed = 0
        success_count_uninformed = 0
        success_count_informed = 0

        for _ in range(num_initial_states):
            problem = FourDominoes(N)
            print('Initial state:')
            problem.show()

            # Uninformed Search
            print('Running Uninformed Search...')
            path, num_expanded, success = uninformed_search(problem, max_expanded_states)
            total_expanded_states_uninformed += num_expanded
            if success:
                success_count_uninformed += 1
                print('Uninformed Search found a solution!')
                print('Path:', path)
            else:
                print('Uninformed Search failed to find a solution within the maximum number of expanded states.')

            # Informed Search
            print('Running Informed Search...')
            path, num_expanded, success = informed_search(problem, max_expanded_states)
            total_expanded_states_informed += num_expanded
            if success:
                success_count_informed += 1
                print('Informed Search found a solution!')
                print('Path:', path)
            else:
                print('Informed Search failed to find a solution within the maximum number of expanded states.')

            print()

        print('--- Results ---')
        print('Uninformed Search:')
        print('Average number of expanded states:', total_expanded_states_uninformed / num_initial_states)
        print('Success rate:', success_count_uninformed / num_initial_states)

        print('Informed Search:')
        print('Average number of expanded states:', total_expanded_states_informed / num_initial_states)
        print('Success rate:', success_count_informed / num_initial_states)
        print()

run_experiments()


-----------------
2x2 grid of tiles
-----------------
Initial state:
╔═══╦═══╗
║╲3╱║╲4╱║
║7╳5║4╳7║
║╱4╲║╱4╲║
╠═══╬═══╣
║╲2╱║╲9╱║
║2╳3║3╳4║
║╱4╲║╱3╲║
╚═══╩═══╝
Running Uninformed Search...
Uninformed Search found a solution!
Path: [((0, 0), (0, 1)), ((0, 0), (1, 0)), ((0, 1), (1, 1))]
Running Informed Search...
Informed Search found a solution!
Path: [((0, 0), (0, 1)), ((0, 0), (1, 0)), ((0, 1), (1, 1))]

Initial state:
╔═══╦═══╗
║╲8╱║╲9╱║
║1╳1║4╳8║
║╱3╲║╱8╲║
╠═══╬═══╣
║╲7╱║╲2╱║
║3╳1║2╳4║
║╱9╲║╱7╲║
╚═══╩═══╝
Running Uninformed Search...
Uninformed Search found a solution!
Path: [((0, 0), (1, 1))]
Running Informed Search...
Informed Search found a solution!
Path: [((0, 0), (1, 1))]

Initial state:
╔═══╦═══╗
║╲3╱║╲9╱║
║8╳4║4╳1║
║╱7╲║╱8╲║
╠═══╬═══╣
║╲7╱║╲8╱║
║1╳2║5╳8║
║╱3╲║╱9╲║
╚═══╩═══╝
Running Uninformed Search...
Uninformed Search found a solution!
Path: [((0, 0), (0, 1)), ((0, 0), (1, 0)), ((0, 0), (1, 1))]
Running Informed Search...
Informed Search found a solution!
Path: [((0, 1), (1

# Report template

## Solution description

In this project, we need to solve the 4-sided dominoes problem by arranging tiles in an NxN grid such that adjacent tiles have the same number in their neighboring sides. I will compare the performance of one uninformed search algorithm and one informed search algorithm. The main highlight of the solution is implementing two different search methods: the Breadth-First Search (BFS) and the A* Search algorithm, both added to the FourDominoes class in our Python solution.

### Search space

The realm I'm exploring is essentially a grid of four-sided domino tiles. Depending on the puzzle's complexity, the grid varies from 2x2 up to 4x4 tiles. The starting point of our journey is a randomly initialized state of domino tiles. The goal is to arrange all tiles so that adjacent sides match in color.

### Successor function

The successor function generates all possible successors of a given state. For each state, we should consider all pairs of tiles that can be swapped, and generate a new state by swapping those tiles. The successor function returns a list of pairs (action, state), where each action represents the swap of two tiles, and the resulting state is the new arrangement after the swap. On average, I found that the number of successors is directly related to the number of dominos in the puzzle.

### Heuristic function

The A* search wouldn't be effective without a heuristic function guiding its steps. I've designed it to calculate the number of mismatched sides in the puzzle. The fewer mismatched sides, the closer we are to the goal. And this heuristic function admissible, it never overestimates the cost to reach the goal because flipping a domino can correct at least one mismatched side.

## Experimental results

This section contains experimental results on the performance of the implemented algorithm on the puzzle. The performance metrics include average number of expanded states, the number of times the goal state was reached, and the number of times the search failed to reach the goal state. I subjected the solution to rigorous testing across a variety of puzzle sizes. Each scenario was run ten times, and we set a limit of 1,000,000 on the number of states that could be expanded in each run. The results were intriguing and shed light on the effectiveness of our solution.

| Puzzle size |&#124;| Uninformed | |         |         |&#124;| Informed | |         |         |
|-------------|------|-------|------|---------|---------|------|-----|------|---------|---------|
|             |&#124;| Avg   | Goal | Fail #1 | Fail #2 |&#124;| Avg | Goal | Fail #1 | Fail #2 |
| 2x2         |&#124;| 1     | 10   | 0       | 0       |&#124;| 1   | 10   | 0       | 0       |
| 3x3         |&#124;| 100   | 3    | 7       | 0       |&#124;| 10  | 7    | 3       | 0       |
| 4x4         |&#124;| 10000 | 0    | 2       | 8       |&#124;| 100 | 3    | 4       | 3       |